In [1]:
import pandas as pd
import numpy as np
from pydataset import data

import pyspark
#Ways to create columns
from pyspark.sql.functions import col, expr
#String
from pyspark.sql.functions import lit #String lit
#String manipulation
from pyspark.sql.functions import regexp_extract, regexp_replace

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 18:04:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [3]:
fav_languages = {"language":["Python", "Java", "Ruby", "JavaScript", "R", "C", "C++"]}

In [4]:
df = pd.DataFrame(fav_languages)

In [5]:
df.shape

(7, 1)

In [6]:
df = spark.createDataFrame(df)

In [7]:
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|      Java|
|      Ruby|
|JavaScript|
|         R|
+----------+
only showing top 5 rows



# 2.Load the mpg dataset as a spark dataframe.

- Create 1 column of output that contains a message like the one below:


`The 1999 audi a4 has a 4 cylinder engine.`
For each vehicle.

In [8]:
mpg = spark.createDataFrame(data("mpg"))

In [9]:
#Print the first line
print(f"The {mpg.first()['year']} {mpg.first()['manufacturer']} {mpg.first()['model']} has a {mpg.first()['cyl']} cylinder engine.")

The 1999 audi a4 has a 4 cylinder engine.


In [10]:
#Another technique
row_looped = mpg.select("year", "manufacturer", "model", "cyl" ).collect()
for rows in row_looped:
    print(f"The {rows[0]} {rows[1]} {rows[2]} has a {rows[3]} cylinder engine.")

The 1999 audi a4 has a 4 cylinder engine.
The 1999 audi a4 has a 4 cylinder engine.
The 2008 audi a4 has a 4 cylinder engine.
The 2008 audi a4 has a 4 cylinder engine.
The 1999 audi a4 has a 6 cylinder engine.
The 1999 audi a4 has a 6 cylinder engine.
The 2008 audi a4 has a 6 cylinder engine.
The 1999 audi a4 quattro has a 4 cylinder engine.
The 1999 audi a4 quattro has a 4 cylinder engine.
The 2008 audi a4 quattro has a 4 cylinder engine.
The 2008 audi a4 quattro has a 4 cylinder engine.
The 1999 audi a4 quattro has a 6 cylinder engine.
The 1999 audi a4 quattro has a 6 cylinder engine.
The 2008 audi a4 quattro has a 6 cylinder engine.
The 2008 audi a4 quattro has a 6 cylinder engine.
The 1999 audi a6 quattro has a 6 cylinder engine.
The 2008 audi a6 quattro has a 6 cylinder engine.
The 2008 audi a6 quattro has a 8 cylinder engine.
The 2008 chevrolet c1500 suburban 2wd has a 8 cylinder engine.
The 2008 chevrolet c1500 suburban 2wd has a 8 cylinder engine.
The 2008 chevrolet c1500 subur

In [11]:
mpg.select(mpg.year, mpg.manufacturer, mpg.model, mpg.cyl).show(1)

+----+------------+-----+---+
|year|manufacturer|model|cyl|
+----+------------+-----+---+
|1999|        audi|   a4|  4|
+----+------------+-----+---+
only showing top 1 row



- Transform the trans column so that it only contains either manual or auto.

In [12]:
mpg.filter(mpg.trans.startswith('m')).show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|a4 quattro|  2.0|2008|  4|manual(m6)|  4| 20| 28|  p|compact|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [13]:
tips = spark.createDataFrame(data("tips"))

In [18]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [24]:
tips.filter(tips.smoker.startswith("Y")).count()

93

4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).